## Luminosity comparison

In [43]:
from import_data import HMXB_parameters_Kaper
from import_data import HMXB_parameters
from import_data import supergiant_stellar_parameters
from import_data import falenga
from functions import scientific_notation
from import_data import stellar_params
from import_data import photometric_params

from astropy.constants import R_sun, L_sun, sigma_sb

import pandas as pd
import dataframe_image as dfi
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.float_format', lambda x: '%.2E' % x)

In [44]:
# Import data
df_hmxb = HMXB_parameters()
df_sg_params = supergiant_stellar_parameters()
df_falenga = falenga()
df_stellar_params = stellar_params()
df_photometric_params = photometric_params()

df_L = pd.DataFrame({'id': df_hmxb['id'].copy(),
                              "L_true": [None] * len(df_hmxb['id']),
                              "L_expected": [None] * len(df_hmxb['id']),
                              "ST_short": [None] * len(df_hmxb['id']),
                              "ST": [None] * len(df_hmxb['id'])})

### Calculate the luminosity from photometric data
This is the luminosity we observe

In [45]:
for i in range(df_hmxb.shape[0]):
    # Spectral type of object
    spectral_type = df_hmxb.loc[i, "spectraltype(short)"]
    # Expected (B-V) of object based on spectral type
    BV0 = df_photometric_params[df_photometric_params['ST'] == spectral_type]["(B-V)0"].reset_index(drop=True).at[0]
    # Observed (B-V) of object based on simbad filters
    BVobs = df_hmxb.loc[i, '(B-V)obs']
    # Bolometric correction (BC)
    BCv = df_photometric_params[df_photometric_params['ST'] == spectral_type]["BCv"].reset_index(drop=True).at[0]

    # Calculate extinction
    Av = 3.2 * (BVobs - BV0)

    # Calculate Absulute magnitude (visual)
    Mv = df_hmxb.loc[i, 'mv'] - 5 * np.log10(df_hmxb.loc[i, 'distance'] * 1000) + 5 - Av

    # Calculate bolomatric absolute magnitude
    Mbol = Mv + BCv

    # Calculate the luminosity in solar luminosities
    L = 10**((Mbol - 4.74) / (-2.5))

    # Put L in luminosity dataframe
    df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "L_true"] = L
    # Put L in the dataframe
    df_hmxb.loc[i, 'luminosity'] = L


### Caclulate the luminosity from the radius and temperature of the optical star
This is the luminosity you would expect

In [46]:
for i in range(df_falenga.shape[0]):
    # Orbital separation
    a = df_falenga.loc[i, 'a'] * R_sun.value
    # Inclination
    inc = df_falenga.loc[i, 'i']
    # Semi eclipse angle
    theta_e = df_falenga.loc[i, 'semi_eclipse_angle']

    # Calculate Radius
    Rob = a * ((np.cos(np.deg2rad(inc)))**2 + (np.sin(np.deg2rad(inc)))**2 * (np.sin(np.deg2rad(theta_e)))**2)**(1/2)

    # Object id
    id = df_falenga.loc[i, 'id']
    # Spectral type
    spectral_type = df_hmxb[df_hmxb["id"] == str(id)]["spectraltype(short)"].reset_index(drop=True).at[0]
    # Spectral temperature
    Teff = df_stellar_params[df_stellar_params["ST"] == spectral_type]["Teff"].reset_index(drop=True).at[0]

    # Calculate Luminosity in solar luminosities
    L = (4 * np.pi * Rob**2 * sigma_sb.value * Teff**4) / L_sun.value

    # Put L in luminosity dataframe
    df_L.loc[df_L["id"] == id, "L_expected"] = L
    df_L.loc[df_L["id"] == id, "ST_short"] = spectral_type
    df_L.loc[df_L["id"] == id, "ST"] = df_hmxb.loc[df_hmxb["id"] == id, "spectraltype"]

### Comparison of $L_{true}$ and $L_{expected}$(bolometric).
- The $L_{expected}$ is calculated from $L = 4 \pi R^{2} \sigma T_{eff}^{4}$, where:
    * $R$ is calculated from the semi-eclipse ange $\theta_{e}$
    * $T_{eff}$ is taken from the models of Martin et al. (2005a)
- The $L_{true}$ is determined from photometric data from Simbad and $(B-V)_{0}$ is from martins et al. (2006)

Calculate $\frac{L_{true}}{L_{expected}}$ (all luminosities are in solar units)

In [47]:
# Filter out all rows with none values
df_L_filtered = df_L.copy().dropna()
df_L_filtered.reset_index(drop=True)

# Calculate fraction
df_L_filtered["L_true/L_expected"] = df_L_filtered["L_true"] / df_L_filtered["L_expected"]

Result

In [48]:
df_L_filtered

,id,L_true,L_expected,ST_short,ST,L_true/L_expected
1,SMC X-1,5.53E+06,2.30E+05,O9I,O9.7Ia+,2.40E+01
2,LMC X-4,3.05E+06,7.62E+04,O8III,O8III,4.00E+01
3,Vela X-1,6.81E+05,7.88E+05,B0I,B0.5Ia,8.65E-01
4,Cen X-3,8.85E+06,1.36E+05,O9III,O9III,6.53E+01
6,4U1538-52,4.19E+05,8.70E+04,B0I,B0.2Ia,4.82E+00
7,4U1700-37,5.50E+05,8.89E+05,O6I,O6Iafcp,6.19E-01


### Comparison of $L_{true}$ and $L_{expected}$(model martins et. al (2005a)).

In [49]:
# Create new dataframe for the luminosities
df_L_model = pd.DataFrame({'id': df_hmxb['id'].copy(),
                              "L_true": df_L["L_true"].copy(),
                              "L_expected": [None] * len(df_hmxb['id']),
                              "ST_short": [None] * len(df_hmxb['id']),
                              "ST": [None] * len(df_hmxb['id'])})

Get the luminosity for the object from the models of Martin et. al (2005a)

In [50]:
for i in range(df_hmxb.shape[0]):
    # Spectral type
    spectral_type = df_hmxb.loc[i, 'spectraltype(short)']
    # Luminosity from model
    logL = df_stellar_params.loc[df_stellar_params["ST"] == spectral_type, "log(L/Lsun)"].reset_index(drop=True).at[0]
    L = 10**logL
    # Save luminosity in dataframe
    df_L_model.loc[i, "L_expected"] = L
    df_L_model.loc[i, "ST_short"] = spectral_type
    df_L_model.loc[i, "ST"] = df_hmxb.loc[i, "spectraltype"]

In [51]:
# Calculate fraction
df_L_model["L_true/L_expected"] = df_L_model["L_true"] / df_L_model["L_expected"]

Result

In [58]:
df_L_model

,id,L_true,L_expected,ST_short,ST,L_true/L_expected
0,2S0114+650,2.06E+05,2.95E+05,B1I,B1Iae,6.99E-01
1,SMC X-1,5.53E+06,4.07E+05,O9I,O9.7Ia+,1.36E+01
2,LMC X-4,3.05E+06,2.24E+05,O8III,O8III,1.36E+01
3,Vela X-1,6.81E+05,3.47E+05,B0I,B0.5Ia,1.97E+00
4,Cen X-3,8.85E+06,1.62E+05,O9III,O9III,5.46E+01
5,GX301-2,6.58E+06,2.95E+05,B1I,B1.5Iaeq,2.23E+01
6,4U1538-52,4.19E+05,3.47E+05,B0I,B0.2Ia,1.21E+00
7,4U1700-37,5.50E+05,6.46E+05,O6I,O6Iafcp,8.53E-01
8,4U1907+09,1.32E+06,4.79E+05,O8I,O8.5Iab,2.76E+00
9,LMC X-1,1.53E+05,4.79E+05,O8I,O8(f)p,3.19E-01


### Comparison of $R_{true}$ and $R_{expected}$
where,
1. $R_{true}$ is calculated from the semi-eclipse angle.
2. $R_{expected}$ is based on the stellar parameters for the stars spectral type. Source Martin et. al (2005a). Get $L$ and $T_{eff}$ and calc R from $L = 4 \pi R^{2} \sigma T_{eff}^{4}$.

Both are calc. in terms of $R_{\odot}$

In [53]:
# Make radius dataframe
df_R = pd.DataFrame({'id': df_hmxb['id'].copy(),
                              "R_true": [None] * len(df_hmxb['id']),
                              "R_expected": [None] * len(df_hmxb['id']),
                              "ST_short": [None] * len(df_hmxb['id']),
                              "ST": [None] * len(df_hmxb['id'])})

1. $R_{true}$

In [54]:
for i in range(df_falenga.shape[0]):
    # Orbital separation
    a = df_falenga.loc[i, 'a'] * R_sun.value
    # Inclination
    inc = df_falenga.loc[i, 'i']
    # Semi eclipse angle
    theta_e = df_falenga.loc[i, 'semi_eclipse_angle']

    # Calculate Radius
    Rob = a * ((np.cos(np.deg2rad(inc)))**2 + (np.sin(np.deg2rad(inc)))**2 * (np.sin(np.deg2rad(theta_e)))**2)**(1/2)

    # Object id
    id = df_falenga.loc[i, 'id']
    # Spectral type
    spectral_type = df_hmxb[df_hmxb["id"] == str(id)]["spectraltype(short)"].reset_index(drop=True).at[0]

    # Put R in luminosity dataframe
    df_R.loc[df_R["id"] == id, "R_true"] = Rob / R_sun.value
    df_R.loc[df_R["id"] == id, "ST_short"] = spectral_type
    df_R.loc[df_R["id"] == id, "ST"] = df_hmxb.loc[df_hmxb["id"] == id, "spectraltype"]

2. $R_{expected}$

In [55]:
for i in range(df_hmxb.shape[0]):
    # Spectral type
    spectral_type = df_hmxb.loc[i, 'spectraltype(short)']

    # Luminosity from model
    logL = df_stellar_params.loc[df_stellar_params["ST"] == spectral_type, "log(L/Lsun)"].reset_index(drop=True).at[0]
    L = 10**logL

    # Effective temperature from model
    Teff = df_stellar_params.loc[df_stellar_params["ST"] == spectral_type, "Teff"].reset_index(drop=True).at[0]

    # Calculate the radius
    R = np.sqrt((L_sun.value / R_sun.value**2) * (L / (4 * np.pi * sigma_sb.value * Teff**4)))

    # Save luminosity in dataframe
    df_R.loc[i, "R_expected"] = R
    df_R.loc[i, "ST_short"] = spectral_type
    df_R.loc[i, "ST"] = df_hmxb.loc[i, "spectraltype"]

Result

In [59]:
df_R_filtered = df_R.copy()
df_R_filtered = df_R_filtered.dropna().reset_index(drop=True)

df_R_filtered

,id,R_true,R_expected,ST_short,ST
0,SMC X-1,1.63E+01,2.16E+01,O9I,O9.7Ia+
1,LMC X-4,7.98E+00,1.37E+01,O8III,O8III
2,Vela X-1,3.38E+01,2.25E+01,B0I,B0.5Ia
3,Cen X-3,1.21E+01,1.32E+01,O9III,O9III
4,4U1538-52,1.12E+01,2.25E+01,B0I,B0.2Ia
5,4U1700-37,2.32E+01,1.98E+01,O6I,O6Iafcp
